In [8]:
import numpy as np
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from statistics import mode
from Neural_Net import boxplots, dataCleaning
from sklearn.model_selection import train_test_split

In [9]:
dataset_temp = pd.read_csv("LBW_Dataset.csv")
dataset_temp = dataCleaning(dataset_temp)
# dataset_temp["Result"] = 0

In [10]:
# features is a dataframe containing all the different features (attributes) of our dataset
features = dataset_temp[['Community','Age','Weight','Delivery phase','HB','IFA','BP','Education','Residence']] # Copy isn't needed lol
# labels is a dataframe containing the corresponding results that we try to predit using the NN
labels = dataset_temp[['Result']]

In [11]:
class NeuralNetwork():
    
    def __init__(self):
        # Seed the random number generator
        np.random.seed(1)
        self.lr = 0.01
        # Set the weights -> 10 hidden layer neurons 
        self.input_hidden_weights = np.random.randn(9, 20)*0.03
        self.output_hidden_weights = np.random.randn(20,1)*0.03
        # Setting the bias 
        self.input_bias = np.random.randn(1, 20)*0.0005
        self.output_bias = np.random.randn(1, 1)*0.0005


    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def tanh(self, x):
        return ((np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x)))
    
    def tanh_derivative(self, x):
        return (1-(self.tanh(x))**2)

    def train(self, X, Y, epochs):
        for epoch in range(epochs):
            X_length = len(X)
            error = 0
            # Pass training set through the neural network row by row
            for x, y in zip(X, Y):
                
                x = np.array([x])
               
                # Layer 1
                # x : input
                # Ah : output
                h = np.dot(x, self.input_hidden_weights) + self.input_bias
                Ah = self.tanh(h)
                
                # Layer 2:
                # Ah : input
                # Yhat : final output
                h2 = np.dot(Ah, self.output_hidden_weights) + self.output_bias
                Yhat = self.sigmoid(h2)
                
                # Calculate the error rate (MSE DERIVATIVE)
                # summ : wi*xi + bi
                # error: d(loss)/d(output)
                # activation error: d(loss)/d(o) * f'(summ)
                # weights: d(loss)/d(w1): d(loss)/d(output) * d(out)/d(summ) * d(summ)/s(wi) : activation error * d(summ)/s(wi) : activation error * input
                # bias: d(loss)/d(bias) : f'(x) * d(summ)/d(b) = f'(x)
                # input: d(loss)/d(input) : d(loss)/d(summ) * d(summ)/d(in) = f'(x) * wi
                error += np.mean(np.square(y-Yhat))
                # derivatives 
                der_error = y-Yhat
                # (der_error) * self.sigmoid_derivative(h2) is the activation error
                act_error_1 = (der_error) * self.sigmoid_derivative(h2)
                backprop_error = np.dot((der_error) * self.sigmoid_derivative(h2), self.output_hidden_weights.T)
                act_error_2 = (backprop_error) * self.tanh_derivative(h)

                # Differential of d(xi*wi + bi)/d(wi) = xi (input to any layer) (Here input to L2 is Ah)
                grad_output_hidden_weights = np.dot(Ah.T, act_error_1)
                # (Here input to L1 is x)
                grad_input_hidden_weights = np.dot(x.T, act_error_2)
                # Differential of d(xi*wi + bi)/d(bi) = 1
                grad_output_bias = act_error_1
                grad_input_bias = act_error_2


                # updation of the weights and biases
                self.output_hidden_weights += self.lr * grad_output_hidden_weights
                self.input_hidden_weights += self.lr * grad_input_hidden_weights
                self.output_bias += self.lr * grad_output_bias
                self.input_bias += self.lr * grad_input_bias
            if(not epoch%100):
                print(epoch, error/X_length)

    def predict(self, X):
        """
        Pass inputs through the neural network to get output
        """
        h = np.dot(X,self.input_hidden_weights) + self.input_bias
        Ah = self.sigmoid(h)
        h2 = np.dot(Ah,self.output_hidden_weights) + self.output_bias
        Yhat = self.sigmoid(h2)
        return Yhat
    
#NOT SURE IF WE SHOULD ADD MORE LAYERS, PLS CHECK IT OUT

In [12]:
if __name__ == "__main__":

    # Initialize the single neuron neural network
    neural_network = NeuralNetwork()
    X = np.array(features, dtype=np.float128)
    Y = np.array(labels, dtype=np.float128)    
    print(X.shape, Y.shape)
    print("Starting Training")
    # split into train test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
    # Train the neural network
    neural_network.train(X_train, Y_train, 200)
    Yhat = neural_network.predict(X_test)
    Y_hat = [1 if i>0.6 else 0 for i in Yhat]
    print(Y_test, Y_hat)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i,j in zip(Y_test,Y_hat):
        if i==1 and j==1:
            TP+=1
        elif i==0 and j==0:
            TN+=1
        elif i==1 and j==0:
            FN+=1
        elif i==0 and j==1:
            FP+=1
            
    accuracy=(TP+TN)/(TP+TN+FP+FN)
    print(f"accuracy : {accuracy}")
    cm=[[0,0],[0,0]]
    cm[0][0]=TN
    cm[0][1]=FP
    cm[1][0]=FN
    cm[1][1]=TP
    # May need to add try except to avoid divide by 0
    p= TP/(TP+FP)
    r= TP/(TP+FN)
    f1=(2*p*r)/(p+r)
    print("Confusion Matrix : ")
    print(cm)
    print("\n")
    print(f"Precision : {p}")
    print(f"Recall : {r}")
    print(f"F1 SCORE : {f1}")

(96, 9) (96, 1)
Starting Training
0 0.21820887502282959739
100 0.16945625843968560988
[[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]] [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy : 0.85
Confusion Matrix : 
[[1, 3], [0, 16]]


Precision : 0.8421052631578947
Recall : 1.0
F1 SCORE : 0.9142857142857143
